# CogneeRetriever

这将帮助您开始使用 Cognee [检索器](/docs/concepts/retrievers)。有关所有 CogneeRetriever 功能和配置的详细文档，请参阅 [API 参考](https://python.langchain.com/api_reference/community/retrievers/langchain_community.retrievers.cognee.CogneeRetriever.html)。

### 集成详情

自带数据（即索引和搜索自定义文档语料库）：

| Retriever | Self-host | Cloud offering | Package |
| :--- | :--- | :---: | :---: |
[CogneeRetriever](https://python.langchain.com/api_reference/community/retrievers/langchain_community.retrievers.cognee.CogneeRetriever.html) | ✅ | ❌ | langchain-cognee |

## 设置

对于 cognee 的默认设置，您唯一需要的是您的 OpenAI API 密钥。

如果你想要从单个查询中获取自动化跟踪信息，你也可以通过取消下面这行注释来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

此检索器存在于 `langchain-cognee` 包中：

In [ ]:
%pip install -qU langchain-cognee

In [ ]:
import nest_asyncio

nest_asyncio.apply()

## 实例化

现在我们可以实例化我们的检索器（retriever）了：

In [ ]:
from langchain_cognee import CogneeRetriever

retriever = CogneeRetriever(
    llm_api_key="sk-",  # OpenAI API Key
    dataset_name="my_dataset",
    k=3,
)

## 用法

添加一些文档，处理它们，然后运行查询。Cognee 会检索与你的查询相关的知识，并生成最终答案。

In [ ]:
# Example of adding and processing documents
from langchain_core.documents import Document

docs = [
    Document(page_content="Elon Musk is the CEO of SpaceX."),
    Document(page_content="SpaceX focuses on rockets and space travel."),
]

retriever.add_documents(docs)
retriever.process_data()

# Now let's query the retriever
query = "Tell me about Elon Musk"
results = retriever.invoke(query)

for idx, doc in enumerate(results, start=1):
    print(f"Doc {idx}: {doc.page_content}")

## 在链中使用

与其它检索器一样，CogneeRetriever 可以通过 [链](/docs/how_to/sequence/) 被整合到 LLM 应用中。

我们需要一个 LLM 或聊天模型：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
from langchain_cognee import CogneeRetriever
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Instantiate the retriever with your Cognee config
retriever = CogneeRetriever(llm_api_key="sk-", dataset_name="my_dataset", k=3)

# Optionally, prune/reset the dataset for a clean slate
retriever.prune()

# Add some documents
docs = [
    Document(page_content="Elon Musk is the CEO of SpaceX."),
    Document(page_content="SpaceX focuses on space travel."),
]
retriever.add_documents(docs)
retriever.process_data()


prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
answer = chain.invoke("What companies do Elon Musk own?")

print("\nFinal chain answer:\n", answer)

## API 参考

TODO：添加 API 参考链接。